In [2]:
import numpy as np
from pathlib import Path

combined_data_filenames = {'Base':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\trb_paper_base_combined.stats.npz'),
                          'No_shift':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\trb_paper_no_shifts_combined.stats.npz'),
                          'Longer':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\trb_paper_longer_visit_combined.stats.npz'),
                          'No_neighbourhood':Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\trb_paper_no_neighbourhoods_combined.stats.npz')}

combined_data = {label:np.load(combined_data_filenames[label]) for label, filename in combined_data_filenames.items()} 

                 
                 

In [21]:
data_names_small = (
    '1.0_customer_customer_weight_over_contacts',
    '1.0_customer_customer_weight_over_agents',
    '1.0_customer_customer_contacts_over_agents',
    '1.0_staff_customer_weight_over_contacts',
    '1.0_staff_customer_weight_over_agents',
    '1.0_staff_customer_contacts_over_agents',
    None,
    '1.0_customer_staff_weight_over_agents',
    '1.0_customer_staff_contacts_over_agents',
)

data_names_middle = (
    '1.5_customer_customer_weight_over_contacts',
    '1.5_customer_customer_weight_over_agents',
    '1.5_customer_customer_contacts_over_agents',
    '1.5_staff_customer_weight_over_contacts',
    '1.5_staff_customer_weight_over_agents',
    '1.5_staff_customer_contacts_over_agents',
    None,
    '1.5_customer_staff_weight_over_agents',
    '1.5_customer_staff_contacts_over_agents',
)

data_names_large = (
    '2.0_customer_customer_weight_over_contacts',    
    '2.0_customer_customer_weight_over_agents',
    '2.0_customer_customer_contacts_over_agents',
    '2.0_staff_customer_weight_over_contacts',
    '2.0_staff_customer_weight_over_agents',
    '2.0_staff_customer_contacts_over_agents',
    None,
    '2.0_customer_staff_weight_over_agents',
    '2.0_customer_staff_contacts_over_agents',
)

data_name_groups = {data_names_middle:'1.5',
                    data_names_small:'1.0',                      
                    data_names_large:'2.0'}

data_name_groups_inv = {value:key for key, value in data_name_groups.items()}

max_string_length = 0
for data_names in data_name_groups.keys():
    for data_name in data_names:
        if data_name is not None and len(data_name) > max_string_length:
            max_string_length = len(data_name)


In [3]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

from matplotlib.colors import to_rgb

row_count = 3
col_count = 3

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')
line_widths = ((3.5,3,2.5,2), (1,1,1,1), (1,1,1,1))
brightness_values = (1, 0.6, 0.6)


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_ax = None
legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    line_widths_data = line_widths[line_style_ind]
    brightness_value = brightness_values[line_style_ind]
    for data_name in data_names:
        if data_name is None:
            legend_ax = axes[ax_ind]
            ax_ind += 1
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = tuple([el*brightness_value for el in to_rgb(colors[label_2_color_ind[label]])])
            line_width = line_widths_data[label_2_color_ind[label]]
            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style,
                                            linewidth=line_width, label=f'{dist_label} - {label}')
            #print(f'{dist_label} - {label} - {data_name} - {len(y)}')
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')
            
            
        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

for ax_ind in range(row_count*col_count):
    col_ind = ax_ind % col_count
    axes[ax_ind].set_xlim(-max_x[col_ind]*0.05, max_x[col_ind])
    axes[ax_ind].grid()
    
    
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels, loc='upper center', ncol=2)
    legend_ax.axis('off')
else:
    leg = plt.legend(legend_handles, legend_labels, loc='upper center', ncol=2)
leg.set_draggable(True)
    
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
%matplotlib widget
from contact_plots import get_cdf_weights_at_probabilities, plot_cdf_weights_at_probabilities
from pathlib import Path
import json

exp_info_filename = Path(r'C:\Users\martijnsparnaa\surfdrive\PhD\Activity scheduler\Scenarios\paper_scenarios\trb_paper_base_experiment.json')

weights_at_probabilities_dict = {}

for data_names, dist_label in data_name_groups.items():
    for data_name in data_names:
        if data_name is None:
            continue
        for label, data in combined_data.items():
            if len(data[data_name]) == 0:
                continue
            weights_at_probabilities = get_cdf_weights_at_probabilities(data[data_name])
            weights_at_probabilities_dict[data_name] = (list(weights_at_probabilities.keys()), list(weights_at_probabilities.values()))
            #print(f'{data_name:>{max_string_length}}: ')
            #for prob, weight in weights_at_probabilities.items():
            #    print(f'{prob:.2f}={weight:.1f}, ', end='')
            #print('')
            
            
with open(exp_info_filename, 'r') as f:
    exp_info_data = json.load(f)
    
array_names_dict = exp_info_data['combined_data_info']['arrayNamesDict']
            
plot_cdf_weights_at_probabilities(weights_at_probabilities_dict, array_names_dict)
                

In [ ]:
print(0 % 3)
print(1 % 3)
print(2 % 3)
print(3 % 3)
print(4 % 3)


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

from _collections import defaultdict
from math import ceil

import numpy as np

from math import inf

STAFF_CUSTOMER = 'staff_customer'
CUSTOMER_CUSTOMER = 'customer_customer'

interval_start = 0
interval_step = 1 #s   
max_interval = interval_step*60 #s
bins = np.arange(interval_start,max_interval, interval_step).tolist()
bins.append(max_interval + interval_step)
hist_data = defaultdict(list)       
    
labels = defaultdict(list)
    
for data_names, dist_label in data_name_groups.items():
    for data_name in data_names:
        if data_name is None or not 'weight_over_contacts' in data_name:
             continue
        if STAFF_CUSTOMER in data_name:              
            interaction_type = STAFF_CUSTOMER
        else:
            interaction_type = CUSTOMER_CUSTOMER                       
        for label, data in combined_data.items():
            contact_data = data[data_name]
            contact_data[contact_data>bins[-1]] = bins[-1]
            
            hist_data[(interaction_type, dist_label)].append(contact_data)
            labels[(interaction_type, dist_label)].append(label)
                    
fig, axes = plt.subplots(len(hist_data), 1)
axes = axes.flatten()

ax_ind = 0
for key, hist_data_label in hist_data.items():
    axes[ax_ind].hist(hist_data_label, bins, label=labels[key])
    axes[ax_ind].set_title(f'{key[0]} - {key[1]}')
    ax_ind += 1
    
leg = plt.legend()
leg.set_draggable(True)

plt.show()


In [10]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

row_count = 3
col_count = 1

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    for data_name in data_names:
        if data_name is None or 'weight_over_contacts' not in data_name:
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = colors[label_2_color_ind[label]]
           
            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style, label=f'{dist_label} - {label}')
            #print(line_handle)
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')

        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

legend_ax = axes[-1]    
    
for ax_ind in range(row_count*col_count):
    col_ind = ax_ind % col_count
    axes[ax_ind].set_xlim(-max_x[col_ind]*0.05, max_x[col_ind])
    
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels)
else:
    leg = plt.legend(legend_handles, legend_labels)
leg.set_draggable(True)
    
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

BadZipFile: Bad magic number for file header

In [9]:
from contact_plots import get_cdf_of_data

%matplotlib widget
import matplotlib.pyplot as plt

row_count = 3
col_count = 1

fig, axes = plt.subplots(row_count, col_count)
axes = axes.flatten()

colors = ('tab:red', 'tab:blue', 'tab:green', 'tab:orange')
line_styles = ('-', '--', ':')


line_style_ind = 0

max_x = {0:0,1:0,2:0}

legend_content = {}
label_2_color_ind = {}

for data_names, dist_label in data_name_groups.items():
    ax_ind = 0
    line_style = line_styles[line_style_ind]
    for data_name in data_names:
        if data_name is None or 'customer_customer' not in data_name:
            continue
        color_ind = 0
        for label, data in combined_data.items():
            if label not in label_2_color_ind:
                label_2_color_ind[label] = color_ind
                color_ind += 1                
            color = colors[label_2_color_ind[label]]
           
            if len(data[data_name]) == 0:
                continue
            x, y = get_cdf_of_data(data[data_name])
            col_ind = ax_ind % col_count
            max_x[col_ind] = max(max_x[col_ind], max(x))
            line_handle = axes[ax_ind].plot(x, y, color=color, linestyle=line_style, label=f'{dist_label} - {label}')
            print(f'{dist_label} - {label} - {data_name} - {len(y)}')
            if (color, line_style) not in legend_content:
                legend_content[(color, line_style)] = (line_handle[0], f'{dist_label} - {label}')

        axes[ax_ind].set_title(data_name[4:])
        ax_ind += 1
    line_style_ind += 1

legend_ax = axes[-1]    
    
   
legend_handles = [value[0] for value in legend_content.values()]
legend_labels = [value[1] for value in legend_content.values()]

if legend_ax is not None:
    leg = legend_ax.legend(legend_handles, legend_labels)
else:
    leg = plt.legend(legend_handles, legend_labels)
leg.set_draggable(True)
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.5 - Base - 1.5_customer_customer_weight_over_contacts - 387537


BadZipFile: Bad magic number for file header

In [20]:
from scipy import stats

labels = combined_data.keys()
max_label_length = max([len(label)for label in labels])
max_entry_length = max(max_label_length + 2, 8)
#print(f'{data_name:>{max_string_length}}: ')


print(f'{" "*max_label_length}   {" "*max_string_length)} ', end='')
for label in labels:
    print(f'{label:>{max_entry_length}} ', end='')
print('')

for data_names, dist_label in data_name_groups.items():
    for data_name in data_names:
        if data_name is None:
            continue
        for label, data in combined_data.items():
            print(f'{label:>{max_label_length}} - {data_name:>{max_string_length}}:', end='')
            for other_label in labels:
                res = stats.cramervonmises_2samp(combined_data[label][data_name], combined_data[other_label][data_name])
                if (abs(res.pvalue - 1.0) < 1e-5 and label == other_label) or abs(res.pvalue) < 1e-5:
                     print(f'{"":{max_entry_length}}', end='')
                else:
                    print(f'{res.pvalue:{max_entry_length}.5f}', end='')
            print('')

                                                                            Base           No_shift             Longer   No_neighbourhood 
            Base - 1.5_customer_customer_weight_over_contacts:                                                                 0.01112
        No_shift - 1.5_customer_customer_weight_over_contacts:                                                                        
          Longer - 1.5_customer_customer_weight_over_contacts:                                                                        
No_neighbourhood - 1.5_customer_customer_weight_over_contacts:           0.01112                                                      
            Base -   1.5_customer_customer_weight_over_agents:                                                                 0.05483
        No_shift -   1.5_customer_customer_weight_over_agents:                                                                        
          Longer -   1.5_customer_customer_weight_o

In [33]:
from scipy import stats

labels = list(data_name_groups_inv.keys())
scen_labels = combined_data.keys()
max_scen_label_length = max([len(label)for label in scen_labels])
max_label_length = max([len(label) for label in labels])
max_entry_length = max(max_label_length + 2, 10)


print(f'{" "*max_label_length}   {" "*max_scen_label_length}   {" "*(max_string_length-4)} ', end='')
for label in labels:
    print(f'{label:>{max_entry_length}} ', end='')
print('')

data_names = data_name_groups_inv[labels[0]]

for ii in range(len(data_names)):
    for label in labels:
        data_name_base = data_name_groups_inv[label][ii]
        if data_name_base is None:
            continue
        for scen_label, data in combined_data.items():
            print(f'{label:>{max_label_length}} - {scen_label:>{max_scen_label_length}} - {data_name_base[4:]:>{(max_string_length-4)}}:', end='')
            for other_label in labels:
                data_name_other = data_name_groups_inv[other_label][ii]
                res = stats.cramervonmises_2samp(data[data_name_base], data[data_name_other])
                print(f'{res.pvalue:{max_entry_length}.4f}', end='')
                #if abs(res.pvalue - 1.0) < 1e-5 or abs(res.pvalue) < 1e-5:
                #     print(f'{"":{max_entry_length}}', end='')
                #else:
                #    print(f'{res.pvalue:{max_entry_length}.4f}', end='')
            print('')

                                                                       1.5        1.0        2.0 
1.5 -             Base - customer_customer_weight_over_contacts:    1.0000    0.0000    0.0000
1.5 -         No_shift - customer_customer_weight_over_contacts:    1.0000    0.0000    0.0000
1.5 -           Longer - customer_customer_weight_over_contacts:    1.0000    0.0000    0.0000
1.5 - No_neighbourhood - customer_customer_weight_over_contacts:    1.0000    0.0000    0.0000
1.0 -             Base - customer_customer_weight_over_contacts:    0.0000    1.0000    0.0000
1.0 -         No_shift - customer_customer_weight_over_contacts:    0.0000    1.0000    0.0000
1.0 -           Longer - customer_customer_weight_over_contacts:    0.0000    1.0000    0.0000
1.0 - No_neighbourhood - customer_customer_weight_over_contacts:    0.0000    1.0000    0.0000
2.0 -             Base - customer_customer_weight_over_contacts:    0.0000    0.0000    1.0000
2.0 -         No_shift - customer_customer_weig

TypeError: 'NoneType' object is not subscriptable